In [1]:
import copy
from typing import Dict, List, Any
import pandas as pd
import json
from collections import defaultdict
import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from datamodel import TradingState, Listing, OrderDepth, Trade, Observation, Order, UserId
%reload_ext autoreload
%autoreload 2

In [3]:
from round_2_ink import Trader
from backtester import Backtester

In [ ]:
def calculate_SQUID_INK_fair(order_depth):
        # assumes order_depth has orders in it
        best_ask = min(order_depth.sell_orders.keys())
        best_bid = max(order_depth.buy_orders.keys())
        filtered_ask = [price for price in order_depth.sell_orders.keys() if abs(order_depth.sell_orders[price]) >= 15]
        filtered_bid = [price for price in order_depth.buy_orders.keys() if abs(order_depth.buy_orders[price]) >= 15]
        mm_ask = min(filtered_ask) if len(filtered_ask) > 0 else best_ask
        mm_bid = max(filtered_bid) if len(filtered_bid) > 0 else best_bid

        mmmid_price = (mm_ask + mm_bid) / 2
        return mmmid_price


def calculate_RAINFOREST_RESIN_fair(order_depth):
        return 10000
listings = {
        'RAINFOREST_RESIN': Listing(symbol='RAINFOREST_RESIN', product='RAINFOREST_RESIN', denomination='SEASHELLS'),
        'SQUID_INK': Listing(symbol='SQUID_INK', product='SQUID_INK', denomination='SEASHELLS'),
        'KELP': Listing(symbol='KELP', product='KELP', denomination='SEASHELLS'),
        'CROISSANTS': Listing(symbol='CROISSANTS', product='CROISSANTS', denomination='SEASHELLS'),
        'JAMS': Listing(symbol='JAMS', product='JAMS', denomination='SEASHELLS'),
        'DJEMBES': Listing(symbol='DJEMBES', product='DJEMBES', denomination='SEASHELLS'),
        'PICNIC_BASKET1': Listing(symbol='PICNIC_BASKET1', product='PICNIC_BASKET1', denomination='SEASHELLS'),
        'PICNIC_BASKET2': Listing(symbol='PICNIC_BASKET2', product='PICNIC_BASKET2', denomination='SEASHELLS'),
    }

position_limit = {
        'RAINFOREST_RESIN': 50,
        'SQUID_INK': 50,
        'KELP': 50,
        'CROISSANTS': 250,
        'JAMS': 350,
        'DJEMBES': 60,
        'PICNIC_BASKET1': 60,
        'PICNIC_BASKET2': 100
    }

fair_calculations = {
        "RAINFOREST_RESIN": calculate_RAINFOREST_RESIN_fair,
        "SQUID_INK": calculate_SQUID_INK_fair
    }
# run
all_backtesters = []
for day in [-1,0,1]:
#day = -1
    market_data = pd.read_csv(f"./round-2-island-data-bottle/prices_round_2_day_{day}.csv", sep=";", header=0)
    trade_history = pd.read_csv(f"./round-2-island-data-bottle/trades_round_2_day_{day}.csv", sep=";", header=0)
    import io
    def _process_data_(file):
        with open(file, 'r') as file:
                log_content = file.read()
        sections = log_content.split('Sandbox logs:')[1].split('Activities log:')
        sandbox_log = sections[0].strip()
        activities_log = sections[1].split('Trade History:')[0]
        # sandbox_log_list = [json.loads(line) for line in sandbox_log.split('\n')]
        trade_history = json.loads(sections[1].split('Trade History:')[1])
        # sandbox_log_df = pd.DataFrame(sandbox_log_list)
        market_data_df = pd.read_csv(io.StringIO(activities_log), sep=";", header=0)
        trade_history_df = pd.json_normalize(trade_history)
        # print(sections[1])
        return market_data_df, trade_history_df
    # market_data, trade_history = _process_data_('./webruns/aggress_time.log')
    # market_data, trade_history = _process_data_('./webruns/null_strategy.log')
    trader = Trader()
    backtester = Backtester(trader, listings, position_limit, fair_calculations, market_data, trade_history, "trade_history_sim.log", False, None)
    backtester.run()
    print(backtester.pnl)
    all_backtesters.append(copy.deepcopy(backtester))


In [ ]:
print(all_backtesters[1].position_history['SQUID_INK'][400:420])

In [ ]:
for day in [0]:
    market_data = pd.read_csv(f"./round-2-island-data-bottle/prices_round_2_day_{day}.csv", sep=";", header=0)
    trade_history = pd.read_csv(f"./round-2-island-data-bottle/trades_round_2_day_{day}.csv", sep=";", header=0)
    i = day + 1
    backtester = all_backtesters[i]
    df_ink = market_data[market_data['product'] == 'SQUID_INK']
    max_pos = max(np.abs(backtester.position_history['SQUID_INK']))
    fig = go.Figure()
    max_pnl = max(backtester.pnl_history['SQUID_INK'])
    # 添加第一条折线（左侧Y轴）
    fig.add_trace(go.Scatter(x=df_ink['timestamp'] / 100, y=np.array(backtester.pnl_history['SQUID_INK'])*50/max_pnl, mode='lines+markers', name='pnl', yaxis='y'))

    # 添加第二条折线（右侧Y轴）
    fig.add_trace(go.Scatter(x=df_ink['timestamp'] / 100, y=np.array(backtester.position_history['SQUID_INK']), mode='lines+markers', name='pos', yaxis='y2'))

    # 添加第三条折线（右侧Y轴）
    fig.add_trace(go.Scatter(x=df_ink['timestamp'] / 100, y=np.array(df_ink['mid_price'])*50/np.nanmax(df_ink['mid_price']), mode='lines+markers', name='price', yaxis='y3'))
    fig.update_layout(
    # 默认的 y 轴（左侧）
    yaxis=dict(
        title='pnl'
    ),
    # 第二个 y 轴（右侧，通常与主轴重叠）
    yaxis2=dict(
        title='pos',
        overlaying='y',  # 与 y 轴重叠
        side='right'     # 放置在右侧
    ),
    # 第三个 y 轴（右侧，位置可以做调整，避免与 yaxis2 重叠）
    yaxis3=dict(
        title='price',
        overlaying='y',  # 与 y 轴重叠
        side='right',
        position=0.85    # 控制该轴在水平方向的位置
    )
    )
    print(max_pnl,max_pos,np.nanmax(df_ink['mid_price']))

    # 显示图形
    fig.show()

In [ ]:
for day in [1]:
    market_data = pd.read_csv(f"./round-2-island-data-bottle/prices_round_2_day_{day}.csv", sep=";", header=0)
    trade_history = pd.read_csv(f"./round-2-island-data-bottle/trades_round_2_day_{day}.csv", sep=";", header=0)
    i = day + 1
    backtester = all_backtesters[i]
    df_ink = market_data[market_data['product'] == 'SQUID_INK']
    max_pos = max(np.abs(backtester.position_history['SQUID_INK']))
    fig = go.Figure()
    max_pnl = max(backtester.pnl_history['SQUID_INK'])
    # 添加第一条折线（左侧Y轴）
    fig.add_trace(go.Scatter(x=df_ink['timestamp'] / 100, y=np.array(backtester.pnl_history['SQUID_INK'])*50/max_pnl, mode='lines+markers', name='pnl', yaxis='y'))

    # 添加第二条折线（右侧Y轴）
    fig.add_trace(go.Scatter(x=df_ink['timestamp'] / 100, y=np.array(backtester.position_history['SQUID_INK']), mode='lines+markers', name='pos', yaxis='y2'))

    # 添加第三条折线（右侧Y轴）
    fig.add_trace(go.Scatter(x=df_ink['timestamp'] / 100, y=np.array(df_ink['mid_price'])*50/np.nanmax(df_ink['mid_price']), mode='lines+markers', name='price', yaxis='y3'))
    fig.update_layout(
    # 默认的 y 轴（左侧）
    yaxis=dict(
        title='pnl'
    ),
    # 第二个 y 轴（右侧，通常与主轴重叠）
    yaxis2=dict(
        title='pos',
        overlaying='y',  # 与 y 轴重叠
        side='right'     # 放置在右侧
    ),
    # 第三个 y 轴（右侧，位置可以做调整，避免与 yaxis2 重叠）
    yaxis3=dict(
        title='price',
        overlaying='y',  # 与 y 轴重叠
        side='right',
        position=0.85    # 控制该轴在水平方向的位置
    )
    )
    print(max_pnl,max_pos,np.nanmax(df_ink['mid_price']))

    # 显示图形
    fig.show()

In [ ]:
np.array(df_ink['mid_price'][1980:2010])

In [ ]:
np.array(np.array(backtester.position_history['SQUID_INK'][1980:2010]))

In [ ]:
np.arr

In [ ]:
all_round3_backtesters = []
listings3 = { 'RAINFOREST_RESIN': Listing(symbol='RAINFOREST_RESIN', product='RAINFOREST_RESIN', denomination='SEASHELLS'),
'SQUID_INK': Listing(symbol='SQUID_INK', product='SQUID_INK', denomination='SEASHELLS'),
 'KELP': Listing(symbol='KELP', product='KELP', denomination='SEASHELLS'),
 'CROISSANTS': Listing(symbol='CROISSANTS', product='CROISSANTS', denomination='SEASHELLS'),
'JAMS': Listing(symbol='JAMS', product='JAMS', denomination='SEASHELLS'),
'DJEMBES': Listing(symbol='DJEMBES', product='DJEMBES', denomination='SEASHELLS'),
 'PICNIC_BASKET1': Listing(symbol='PICNIC_BASKET1', product='PICNIC_BASKET1', denomination='SEASHELLS'),
'PICNIC_BASKET2': Listing(symbol='PICNIC_BASKET2', product='PICNIC_BASKET2', denomination='SEASHELLS'),

'VOLCANIC_ROCK': Listing(symbol='VOLCANIC_ROCK', product='VOLCANIC_ROCK', denomination='SEASHELLS'),
'VOLCANIC_ROCK_VOUCHER_9500': Listing(symbol='VOLCANIC_ROCK_VOUCHER_9500', product='VOLCANIC_ROCK_VOUCHER_9500', denomination='SEASHELLS'),
'VOLCANIC_ROCK_VOUCHER_9750': Listing(symbol='VOLCANIC_ROCK_VOUCHER_9750', product='VOLCANIC_ROCK_VOUCHER_9750', denomination='SEASHELLS'),
 'VOLCANIC_ROCK_VOUCHER_10000': Listing(symbol='VOLCANIC_ROCK_VOUCHER_10000', product='VOLCANIC_ROCK_VOUCHER_10000', denomination='SEASHELLS'),
 'VOLCANIC_ROCK_VOUCHER_10250': Listing(symbol='VOLCANIC_ROCK_VOUCHER_10250', product='VOLCANIC_ROCK_VOUCHER_10250', denomination='SEASHELLS'),
 'VOLCANIC_ROCK_VOUCHER_10500': Listing(symbol='VOLCANIC_ROCK_VOUCHER_10500', product='VOLCANIC_ROCK_VOUCHER_10500', denomination='SEASHELLS'),
 }

position_limit3 = {
'RAINFOREST_RESIN': 50,
'SQUID_INK': 50,
'KELP': 50,
'CROISSANTS': 250,
'JAMS': 350,
'DJEMBES': 60,
'PICNIC_BASKET1': 60,
'PICNIC_BASKET2': 100,
'VOLCANIC_ROCK_VOUCHER_9500': 200,
'VOLCANIC_ROCK_VOUCHER_9750': 200,
 'VOLCANIC_ROCK_VOUCHER_10000': 200,
}

for day in [0,1,2]:
#day = -1
    market_data = pd.read_csv(f"./round-3-island-data-bottle/prices_round_3_day_{day}.csv", sep=";", header=0)
    trade_history = pd.read_csv(f"./round-3-island-data-bottle/trades_round_3_day_{day}.csv", sep=";", header=0)
    df_ink = market_data[market_data['product'] == 'SQUID_INK']
    print(np.where(np.isnan(df_ink['bid_volume_1'])))
    import io
    def _process_data_(file):
        with open(file, 'r') as file:
                log_content = file.read()
        sections = log_content.split('Sandbox logs:')[1].split('Activities log:')
        sandbox_log = sections[0].strip()
        activities_log = sections[1].split('Trade History:')[0]
        # sandbox_log_list = [json.loads(line) for line in sandbox_log.split('\n')]
        trade_history = json.loads(sections[1].split('Trade History:')[1])
        # sandbox_log_df = pd.DataFrame(sandbox_log_list)
        market_data_df = pd.read_csv(io.StringIO(activities_log), sep=";", header=0)
        trade_history_df = pd.json_normalize(trade_history)
        # print(sections[1])
        return market_data_df, trade_history_df
    # market_data, trade_history = _process_data_('./webruns/aggress_time.log')
    # market_data, trade_history = _process_data_('./webruns/null_strategy.log')
    trader = Trader()

    backtester = Backtester(trader, listings3, position_limit3, {}, market_data, trade_history, "trade_history_sim.log", False, None)
    backtester.run()
    print(backtester.pnl)
    all_round3_backtesters.append(copy.deepcopy(backtester))